Try to learn (1) as a piecewise constant curve

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib_inline.backend_inline import set_matplotlib_formats
import os
from itertools import chain

set_matplotlib_formats("pdf", "svg")

from deepthermal.FFNN_model import fit_FFNN, FFNN, init_xavier
from deepthermal.validation import (
    create_subdictionary_iterator,
    k_fold_cv_grid,
    add_dictionary_iterators,
)
from deepthermal.plotting import plot_result, plot_model_1d

from neural_reparam.reparametrization import (
    get_elastic_metric_loss,
    compute_loss_reparam,
)
from neural_reparam.interpolation import get_pc_curve
import experiments.curves as c1

In [ ]:
########
DIR = "../figures/curve_1_pc/"
SET_NAME = "eks_2"
PATH_FIGURES = os.path.join(DIR, SET_NAME)
if not os.path.exists(PATH_FIGURES):
    os.makedirs(PATH_FIGURES)
########


FOLDS = 1
N = 128  # training points internal
lr_scheduler = lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, mode="min", factor=0.5, patience=10, verbose=True
)
loss_func = get_elastic_metric_loss(
    r=get_pc_curve(c1.r, N), constrain_cost=1e3, verbose=False
)
penalty_free_loss_func = get_elastic_metric_loss(
    r=get_pc_curve(c1.r, N), constrain_cost=0, verbose=False
)

MODEL_PARAMS = {
    "input_dimension": [1],
    "output_dimension": [1],
    "activation": ["relu"],
    "n_hidden_layers": [2],  # ,8,16,64],
    "neurons": [32],
}

# extend the previous dict with the zip of this
MODEL_PARAMS_EXPERIMENT = {
    "model": [FFNN],
}
TRAINING_PARAMS = {
    "batch_size": [N // 2],
    "regularization_param": [0],
    "compute_loss": [compute_loss_reparam],
    "loss_func": [loss_func],
}
# extend the previous dict with the zip of this
TRAINING_PARAMS_EXPERIMENT = {
    "optimizer": ["strong_wolfe"],
    "num_epochs": [100],
    "learning_rate": [1],
    "lr_scheduler": [lr_scheduler],
}

In [ ]:
# create iterators
model_params_iter = create_subdictionary_iterator(MODEL_PARAMS)

model_exp_iter = create_subdictionary_iterator(MODEL_PARAMS_EXPERIMENT, product=False)
exp_model_params_iter = add_dictionary_iterators(model_exp_iter, model_params_iter)

training_params_iter = create_subdictionary_iterator(TRAINING_PARAMS)
training_exp_iter = create_subdictionary_iterator(
    TRAINING_PARAMS_EXPERIMENT, product=False
)
exp_training_params_iter = add_dictionary_iterators(
    training_exp_iter, training_params_iter
)

In [ ]:
# Load data
x_train = torch.linspace(0, 1, N, requires_grad=True).unsqueeze(1)
q_pl = get_pc_curve(c1.q, N)
q_train = q_pl(x_train.detach())

data = TensorDataset(x_train, q_train)

Do the actual training

In [ ]:
cv_results = k_fold_cv_grid(
    model_params=exp_model_params_iter,
    fit=fit_FFNN,
    training_params=exp_training_params_iter,
    data=data,
    folds=FOLDS,
    verbose=True,
    trials=10,
)

In [ ]:
# plotting

disc_points = x_train.detach()

plot_kwargs = {
    "x_test": disc_points,
    "x_train": disc_points,
    "y_train": c1.ksi(disc_points),
    "x_axis": "t",
    "y_axis": "$\\varphi(t)$",
    "compare_label": "analytical solution (smooth)",
}
plot_result(
    path_figures=PATH_FIGURES,
    **cv_results,
    plot_function=plot_model_1d,
    function_kwargs=plot_kwargs,
)